In [14]:
## Task: 
## (1) Which keywords can let your comments get a higher score in Reddit? 
## 2. Compared the run time of (1) in common way and pyspark
import time
import pandas as pd
from pyspark.sql import SparkSession
from operator import add
import os
from pyspark.sql.functions import length
from Pre_processing import data  # import data from Pre_processing

In [15]:
# preparation for Task(1) and (2)
start_time = time.time()
stop_words = ['a', 'an', 'and', 'are', 'as', 'at', 'be', 'but', 'by', 'for', 'if', 'in', 'into', 'is', 'it',
              'no', 'not', 'of', 'on', 'or', 'such', 'that', 'the', 'their', 'then', 'there', 'these', 'they',
              'this', 'to', 'was', 'will', 'with', '!', '@', '#', '$', '%', '^', '&', '*', '(', ')', '-', 
              '_', '+', '=', '[', ']', '{', '}', ';', ':', '"', "'", '<', '>', ',', '.', '/', '?', '\\', 
              '|', '`', '~', '']

In [21]:
def task_function(data, use_pyspark=False):
    start_time = time.time()
    if use_pyspark:
        print('Using Pyspark')
        spark_session = SparkSession.builder\
        .master("spark://192.168.2.70:7077") \
        .appName("yufengchen_app")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.dynamicAllocation.shuffleTracking.enabled",True)\
        .config("spark.shuffle.service.enabled", True)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.cores.max", 4)\
        .getOrCreate()

        data_pyspark = spark_session.createDataFrame(data)
        data_pyspark = data_pyspark.filter((length(data_pyspark.body) >= 10) & (data_pyspark['body'] != '[deleted]'))
        data = data_pyspark.select('score', 'body')
        word_score = {}
        word_count = {}
        data_collect = data.collect()
        for row in data_collect:
            score = row['score']
            words_list = row['body'].split()
            for word in words_list:
                # filter stop word
                if word not in stop_words:
                    if word not in word_count.keys():
                        word_count[word] = 1
                        word_score[word] = score
                    else:
                        word_count[word] = 1 + word_count[word]
                        word_score[word] = score + word_score[word]
        
    else:
        print('Without using Pyspark')
        data = data.loc[:, ['score', 'body']]
        data = data[data['body'].str.len() >= 10] # drop len(body) less than 10 
        data = data.drop(data[data['body'] == '[deleted]'].index)  # drop body that deleted
        word_score = {}
        word_count = {}
        
        for index, row in data.iterrows():
            score = row['score']
            words_list = row['body'].split()
            for word in words_list:
                # filter stop word
                if word not in stop_words:
                    if word not in word_count.keys():
                        word_count[word] = 1
                        word_score[word] = score
                    else:
                        word_count[word] = 1 + word_count[word]
                        word_score[word] = score + word_score[word]

    # filter the word appear less than 5 times
    filtered_word_count = {k: v for k, v in word_count.items() if v >= 5}
    filtered_word_score = {k: v for k, v in word_score.items() if k in filtered_word_count.keys()}
    
    for word,times in filtered_word_count.items():
        if filtered_word_score[word] != 0:
            filtered_word_score[word] = filtered_word_score[word]/times
        else:
            pass    

    sorted_filtered_word_score = dict(sorted(filtered_word_score.items(), key=lambda x: x[1], reverse=True))
    # top 10 word  that may let your comments get a higher score in Reddit
    for key, value in list(sorted_filtered_word_score.items())[:10]:
        print(f"{key}: {value}")

    end_time = time.time()
    if use_pyspark:
        print('Spend:', end_time-start_time,'s')
    else:
        print('Spend:', end_time-start_time,'s')

In [22]:
task_function(data, use_pyspark=True)

Using Pyspark


Viaweb: 19.5
books.: 11.5
money,: 10.5
spoken: 9.6
Stossel: 9.4
about,: 9.2
loved: 8.833333333333334
efficient: 8.8
You've: 8.6
technical: 8.5
Spend: 5.218137502670288 s


In [23]:
task_function(data)

Without using Pyspark
Viaweb: 19.5
books.: 11.5
money,: 10.5
spoken: 9.6
Stossel: 9.4
about,: 9.2
loved: 8.833333333333334
efficient: 8.8
You've: 8.6
technical: 8.5
Spend: 0.25510096549987793 s
